In [ ]:
import pandas as pd
import keras
from keras.layers import LSTM, SimpleRNN, Input, Bidirectional, TimeDistributed, Dropout, Dense, Activation, BatchNormalization
from keras.models import Model
from keras.utils.np_utils import to_categorical
from keras.regularizers import l2
import tensorflow as tf
import numpy as np
import os
import fnmatch
from matplotlib import pyplot as plt
from IPython.display import clear_output

## Model

In [ ]:
def get_model(input_shape, output_shape, regularizer, rs=0, lr = 0.00001, path = None):
    inputs = Input(shape=input_shape)
    X = Dropout(0.2)(inputs)
    X = Dense(4096, activation='relu', kernel_regularizer=regularizer(rs))(X)
#    X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    X = Dense(2048, activation='relu', kernel_regularizer=regularizer(rs))(X)
    X = Dense(2048, activation='relu', kernel_regularizer=regularizer(rs))(X)
    X = Bidirectional(LSTM(1024, return_sequences=True, kernel_regularizer=regularizer(rs)))(X)
    outputs = TimeDistributed(Dense(output_shape, activation='softmax'))(X)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=tf.train.AdamOptimizer(lr),
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'],
    )
    
    if path != None:
        model.load_weights(path)
    
    return model

## Callbacks

In [ ]:
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.acc = []
        self.val_losses = []
        self.val_acc = []
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        f, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=False)
        self.ax1 = ax1
        self.ax2 = ax2
    
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.acc.append(logs.get('categorical_accuracy'))
        self.val_losses.append(logs.get('val_loss'))
        self.val_acc.append(logs.get('val_categorical_accuracy'))
        self.i += 1
        
        clear_output(wait=True)
        self.ax1.plot(self.x, self.losses, label="loss")
        self.ax1.plot(self.x, self.val_losses, label="val loss")
        self.ax1.legend()
        self.ax2.plot(self.x, self.acc, label="accuracy")
        self.ax2.plot(self.x, self.val_acc, label="val accuracy")
        self.ax2.legend()
        plt.show()
        
        for i in range(self.i):
            print('Epoch ' + str(i+1))
            print('-----------------------')
            print('- Loss:', self.losses[i])
            print('- Accuracy:', self.acc[i])
            print('- Validation loss:', self.val_losses[i])
            print('- Validation accuracy:', self.val_acc[i])
            print(' ')
            
checkpoint = keras.callbacks.ModelCheckpoint(
    './convnet_weights/lstm/weights.{epoch:02d}-{val_loss:.2f}.hdf5', 
    monitor='val_loss',
    verbose=0, 
    save_best_only=False, 
    save_weights_only=True, 
    mode='max', 
    period=1,
)

## Data

In [ ]:
def generator(path, mode, num_batches, random=True):
    counter = 0
    indices = None
    if random == True:
        indices = np.random.permutation(list(range(num_batches)))
    else:
        indices = list(range(num_batches))
    
    while True:
        if counter >= num_batches:
            counter = 0
            if random == True:
                indices = np.random.permutation(list(range(num_batches)))

        features = np.load(path + 'features_' + mode + '_' + str(indices[counter]) + '.npy')
        classes = np.load(path + 'classes_' + mode + '_' + str(indices[counter]) + '.npy')
        
        counter += 1
        yield features, classes
        
def get_metadata(path):
    features = np.load(path + 'features_train_0.npy')
    input_shape = features.shape[1:]
    classes = np.load(path + 'classes_train_0.npy')
    num_classes = classes.shape[2]
    steps_per_epoch = len(fnmatch.filter(os.listdir(path),'*features_train_*'))
    validation_steps = len(fnmatch.filter(os.listdir(path),'*features_dev_*'))
    return input_shape, num_classes, steps_per_epoch, validation_steps

def get_classes(mode, steps, classnames_make, classnames_model):
    classes = None
    for i in range(steps):
        cl = np.load('./features/lstm/2_steps/classes_' + mode + '_' + str(i) + '.npy')
        if classes is None:
            classes = cl
        else:
            classes = np.append(classes, cl, axis=0)
        
    classes = np.argmax(classes, axis=2)
    classnames = list(map(lambda p: [classnames_make[p[0]], classnames_model[p[1]]], classes))
    return classes, classnames

In [ ]:
input_shape, num_classes, steps_per_epoch, validation_steps = get_metadata('./features/lstm/2_steps/')

In [ ]:
print('Input shape:', input_shape)
print('Number of classes:', num_classes)
print('Steps per epoch:', steps_per_epoch) 
print('Validation steps:', validation_steps)

In [ ]:
labels = pd.read_csv('./stanford-car-dataset-by-classes-folder/labels.csv')
classnames_make = labels['make'].unique()
classnames_model = labels['model'].unique()
classnames_make.sort()
classnames_model.sort()

## Training

In [ ]:
model = get_model(
    input_shape, 
    num_classes, 
    regularizer=l2,
#    rs=0.1,
    lr=0.000001,
    path='./convnet_weights/lstm/weights.150-1.13.hdf5',
)
plot_losses = PlotLosses()

In [ ]:
model.summary()

In [ ]:
model.fit_generator(
    generator('./features/lstm/2_steps/', 'train', steps_per_epoch),
    steps_per_epoch=steps_per_epoch,
    epochs=150,
    validation_data=generator('./features/lstm/2_steps/', 'dev', validation_steps),
    validation_steps=validation_steps,
    callbacks=[plot_losses, checkpoint],
)

## Encoding

In [ ]:
classes_dev, classnames_dev = get_classes('dev', validation_steps, classnames_make, classnames_model)
classes_train, classnames_train = get_classes('train', steps_per_epoch, classnames_make, classnames_model)

In [ ]:
classnames_dev = np.array(list(map(lambda cl: cl[0] + ' ' + cl[1], classnames_dev)))
classnames_train = np.array(list(map(lambda cl: cl[0] + ' ' + cl[1], classnames_train)))
classes_dev = pd.get_dummies(classnames_dev)
classes_train = pd.get_dummies(classnames_train)
print(classes_dev.shape)
print(classes_train.shape)

In [ ]:
np.save('./features/lstm/encoded/classes_dev.npy', classes_dev)
np.save('./features/lstm/encoded/classes_train.npy', classes_train)
np.save('./features/lstm/encoded/classnames_dev.npy', classnames_dev)
np.save('./features/lstm/encoded/classnames_train.npy', classnames_train)

In [ ]:
model = get_model(
    input_shape, 
    num_classes, 
    regularizer=l2,
    lr=0.00001,
    path='./convnet_weights/lstm/weights.19-1.13.hdf5',
)

In [ ]:
model.summary()

In [ ]:
encoder = Model(inputs=model.input, outputs=model.layers[6].output)

In [ ]:
encoded_features_dev = encoder.predict_generator(
    generator('./features/lstm/2_steps/', 'dev', validation_steps, random=False),
    steps=validation_steps,
    verbose=True,
)

In [ ]:
encoded_features_train = encoder.predict_generator(
    generator('./features/lstm/2_steps/', 'train', steps_per_epoch, random=False),
    steps=steps_per_epoch,
    verbose=True,
)

In [ ]:
print(encoded_features_dev.shape)
print(encoded_features_train.shape)

In [ ]:
np.save('./features/lstm/encoded/features_dev.npy', encoded_features_dev)
np.save('./features/lstm/encoded/features_train.npy', encoded_features_train)

## Predictions

In [ ]:
correct_classes, correct_classnames = get_classes('dev', validation_steps, classnames_make, classnames_model)

In [ ]:
correct_classes

In [ ]:
correct_classnames

In [ ]:
model = get_model(
    input_shape, 
    num_classes,
    regularizer=l2,
    lr=0.00001,
    path='./convnet_weights/lstm/weights.19-1.13.hdf5',
)

In [ ]:
predictions = model.predict_generator(
    generator('./features/lstm/2_steps/', 'dev', validation_steps, random=False),
    steps=validation_steps,
    verbose=True,
)

In [ ]:
predictions = np.argmax(predictions, axis=2)

In [ ]:
predictions

In [ ]:
prediction_names = list(map(lambda p: [classnames_make[p[0]], classnames_model[p[1]]], predictions))

In [ ]:
prediction_names

In [ ]:
predicted_make = predictions[:, 0]
predicted_model = predictions[:, 1]
true_make = correct_classes[:, 0]
true_model = correct_classes[:, 1]

In [ ]:
false_make = predicted_make[predicted_make != true_make]
false_make.shape

In [ ]:
false_model = predicted_model[predicted_model != true_model]
false_model.shape

In [ ]:
false_predictions = predictions[(predictions[:,0] != correct_classes[:,0]) | (predictions[:,1] != correct_classes[:,1])]
false_predictions.shape